# Лабораторная работа №3

## Реализация сверточной нейронной сети

В работе предлагается использовать набор данных _notMNIST_, который состоит из изображений размерностью 28×28 первых 10 букв латинского алфавита (_A_ ... _J_, соответственно). Обучающая выборка содержит порядка 500 тыс. изображений, а тестовая – около 19 тыс.

Данные можно скачать по ссылке:

* https://commondatastorage.googleapis.com/books1000/notMNIST_large.tar.gz (большой набор данных);

* https://commondatastorage.googleapis.com/books1000/notMNIST_small.tar.gz (маленький набор данных);

Описание данных на английском языке доступно по ссылке:
http://yaroslavvb.blogspot.sg/2011/09/notmnist-dataset.html

### Задание 1

Реализуйте нейронную сеть с двумя сверточными слоями, и одним полносвязным с нейронами с кусочно-линейной функцией активации. Какова точность построенной модели?

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [0]:
BASE_DIR = '/content/drive/My Drive/Colab Files/mo-2'

import sys

sys.path.append(BASE_DIR)

import os

os.chdir(BASE_DIR)

In [0]:
import pandas as pd

dataframe = pd.read_pickle("./large.pkl")

In [4]:
! pip install tensorflow-gpu --pre --quiet

! pip show tensorflow-gpu

Name: tensorflow-gpu
Version: 2.2.0rc2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: tensorboard, scipy, wrapt, google-pasta, tensorflow-estimator, h5py, keras-preprocessing, six, gast, astunparse, numpy, termcolor, absl-py, wheel, opt-einsum, grpcio, protobuf
Required-by: 


In [0]:
import tensorflow as tf

In [0]:
# To fix memory leak: https://github.com/tensorflow/tensorflow/issues/33009

tf.compat.v1.disable_eager_execution()

In [0]:
import numpy as np

In [8]:
x = np.asarray(list(dataframe['data']))[..., np.newaxis]

x = tf.keras.utils.normalize(x, axis = 1)

x.shape

(461946, 28, 28, 1)

In [0]:
IMAGE_DIM_0, IMAGE_DIM_1 = x.shape[1], x.shape[2]

In [10]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(dataframe['label'].astype('category').cat.codes.astype('int32'))

y.shape

(461946, 10)

In [0]:
CLASSES_N = y.shape[1]

In [0]:
DENSE_LAYER_WIDTH = 5000

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten

model = tf.keras.Sequential()

model.add(Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMAGE_DIM_0, IMAGE_DIM_1, 1)))
model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(Flatten())
model.add(Dense(DENSE_LAYER_WIDTH, activation='relu'))
model.add(Dense(CLASSES_N))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
def cat_cross_from_logits(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred, from_logits = True)

model.compile(optimizer = 'sgd',
              loss = cat_cross_from_logits,
              metrics = ['categorical_accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        4640      
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 5000)              125445000 
_________________________________________________________________
dense_1 (Dense)              (None, 10)                50010     
Total params: 125,499,810
Trainable params: 125,499,810
Non-trainable params: 0
_________________________________________________________________


In [0]:
BATCH_SIZE = 128

In [0]:
r = 3608

In [0]:
VAL_SPLIT_RATE = 0.1

In [0]:
EPOCHS_N = 20

In [20]:
model.fit(x = x[:r * BATCH_SIZE], y = y[:r * BATCH_SIZE], epochs = EPOCHS_N, batch_size = BATCH_SIZE,
          validation_split = VAL_SPLIT_RATE)

Train on 415641 samples, validate on 46183 samples
Epoch 1/20
415641/415641 [==============================] - 67s 161us/sample - loss: 0.5828 - categorical_accuracy: 0.8328 - val_loss: 3.6818 - val_categorical_accuracy: 0.0402
Epoch 2/20
415641/415641 [==============================] - 67s 161us/sample - loss: 0.3997 - categorical_accuracy: 0.8802 - val_loss: 3.1953 - val_categorical_accuracy: 0.2366
Epoch 3/20
415641/415641 [==============================] - 67s 160us/sample - loss: 0.3576 - categorical_accuracy: 0.8924 - val_loss: 2.8583 - val_categorical_accuracy: 0.3743
Epoch 4/20
415641/415641 [==============================] - 67s 160us/sample - loss: 0.3310 - categorical_accuracy: 0.9001 - val_loss: 3.6683 - val_categorical_accuracy: 0.1414
Epoch 5/20
415641/415641 [==============================] - 67s 162us/sample - loss: 0.3099 - categorical_accuracy: 0.9059 - val_loss: 3.5571 - val_categorical_accuracy: 0.1573
Epoch 6/20
415641/415641 [==============================] - 67s 

### Задание 2

Замените один из сверточных слоев на слой, реализующий операцию пулинга (_Pooling_) с функцией максимума или среднего. Как это повлияло на точность классификатора?

In [0]:
from tensorflow.keras.layers import MaxPooling2D

model_2 = tf.keras.Sequential()

model_2.add(Conv2D(16, 3, padding = 'same', activation = 'relu', input_shape = (IMAGE_DIM_0, IMAGE_DIM_1, 1)))
model_2.add(MaxPooling2D())
model_2.add(Flatten())
model_2.add(Dense(DENSE_LAYER_WIDTH, activation = 'relu'))
model_2.add(Dense(CLASSES_N))

In [0]:
model_2.compile(optimizer = 'sgd',
               loss = cat_cross_from_logits,
               metrics = ['categorical_accuracy'])

In [23]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 5000)              15685000  
_________________________________________________________________
dense_3 (Dense)              (None, 10)                50010     
Total params: 15,735,170
Trainable params: 15,735,170
Non-trainable params: 0
_________________________________________________________________


In [24]:
model_2.fit(x = x[:r * BATCH_SIZE], y = y[:r * BATCH_SIZE], epochs = EPOCHS_N, batch_size = BATCH_SIZE,
            validation_split = VAL_SPLIT_RATE)

Train on 415641 samples, validate on 46183 samples
Epoch 1/20
415641/415641 [==============================] - 17s 41us/sample - loss: 0.7566 - categorical_accuracy: 0.7999 - val_loss: 6.3352 - val_categorical_accuracy: 0.0000e+00
Epoch 2/20
415641/415641 [==============================] - 17s 41us/sample - loss: 0.5386 - categorical_accuracy: 0.8429 - val_loss: 5.3920 - val_categorical_accuracy: 0.0000e+00
Epoch 3/20
415641/415641 [==============================] - 17s 40us/sample - loss: 0.4687 - categorical_accuracy: 0.8610 - val_loss: 4.6387 - val_categorical_accuracy: 0.0000e+00
Epoch 4/20
415641/415641 [==============================] - 17s 41us/sample - loss: 0.4241 - categorical_accuracy: 0.8732 - val_loss: 3.5897 - val_categorical_accuracy: 0.0688
Epoch 5/20
415641/415641 [==============================] - 18s 42us/sample - loss: 0.3944 - categorical_accuracy: 0.8823 - val_loss: 3.9220 - val_categorical_accuracy: 0.0213
Epoch 6/20
415641/415641 [==============================]

### Задание 3

Реализуйте классическую архитектуру сверточных сетей _LeNet-5_ (http://yann.lecun.com/exdb/lenet/).

In [0]:
from tensorflow.keras.layers import AveragePooling2D

model_3 = tf.keras.Sequential()

model_3.add(Conv2D(6, kernel_size = (5, 5), strides = (1, 1), activation = 'tanh', padding = 'same',
                   input_shape = (IMAGE_DIM_0, IMAGE_DIM_1, 1)))
model_3.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
model_3.add(Conv2D(16, kernel_size = (5, 5), strides = (1, 1), activation = 'tanh', padding = 'valid'))
model_3.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
model_3.add(Flatten())
model_3.add(Dense(120, activation = 'tanh'))
model_3.add(Dense(84, activation = 'tanh'))
model_3.add(Dense(CLASSES_N, activation = 'softmax'))

In [0]:
model_3.compile(optimizer = 'adam',
                loss = 'categorical_crossentropy',
                metrics = ['categorical_accuracy'])

In [27]:
model_3.fit(x = x[:r * BATCH_SIZE], y = y[:r * BATCH_SIZE], epochs = EPOCHS_N, batch_size = BATCH_SIZE,
            validation_split = VAL_SPLIT_RATE)

Train on 415641 samples, validate on 46183 samples
Epoch 1/20
415641/415641 [==============================] - 11s 27us/sample - loss: 0.4755 - categorical_accuracy: 0.8556 - val_loss: 3.9593 - val_categorical_accuracy: 0.0066
Epoch 2/20
415641/415641 [==============================] - 11s 26us/sample - loss: 0.3540 - categorical_accuracy: 0.8905 - val_loss: 2.9415 - val_categorical_accuracy: 0.1864
Epoch 3/20
415641/415641 [==============================] - 11s 26us/sample - loss: 0.3196 - categorical_accuracy: 0.9010 - val_loss: 2.4161 - val_categorical_accuracy: 0.4554
Epoch 4/20
415641/415641 [==============================] - 11s 26us/sample - loss: 0.2977 - categorical_accuracy: 0.9070 - val_loss: 2.3002 - val_categorical_accuracy: 0.5382
Epoch 5/20
415641/415641 [==============================] - 11s 26us/sample - loss: 0.2841 - categorical_accuracy: 0.9113 - val_loss: 2.6019 - val_categorical_accuracy: 0.4619
Epoch 6/20
415641/415641 [==============================] - 11s 27us/

### Задание 4

Сравните максимальные точности моделей, построенных в лабораторных работах 1-3. Как можно объяснить полученные различия?